In [17]:
import numpy as np
import pandas as pd
colnames=['Date','Temperature','Humidity','Light','CO2','HumidityRatio','Occupancy']
df=pd.read_csv(r"C:\Users\rajan\Downloads\occupancy_data\datatraining.txt")
df.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
1,2015-02-04 17:51:00,23.18,27.2720,426.0,721.25,0.004793,1
2,2015-02-04 17:51:59,23.15,27.2675,429.5,714.00,0.004783,1
3,2015-02-04 17:53:00,23.15,27.2450,426.0,713.50,0.004779,1
4,2015-02-04 17:54:00,23.15,27.2000,426.0,708.25,0.004772,1
5,2015-02-04 17:55:00,23.10,27.2000,426.0,704.50,0.004757,1


In [34]:
df_1=df.iloc[:, 1:6]

In [35]:
df_1.describe()

,Temperature,Humidity,Light,CO2,HumidityRatio
count,8143.000000,8143.000000,8143.000000,8143.000000,8143.000000
mean,20.619084,25.731507,119.519375,606.546243,0.003863
std,1.016916,5.531211,194.755805,314.320877,0.000852
min,19.000000,16.745000,0.000000,412.750000,0.002674
25%,19.700000,20.200000,0.000000,439.000000,0.003078
50%,20.390000,26.222500,0.000000,453.500000,0.003801
75%,21.390000,30.533333,256.375000,638.833333,0.004352
max,23.180000,39.117500,1546.333333,2028.500000,0.006476


In [36]:
df_1= df_1.dropna()

In [37]:
df_low=df_1[df_1.Humidity<26.222500]
df_high=df_1[df_1.Humidity>=26.222500]

In [38]:
df_low

,Temperature,Humidity,Light,CO2,HumidityRatio
193,21.500000,26.20,0.0,502.0,0.004153
194,21.566667,26.20,0.0,502.0,0.004170
195,21.500000,26.20,0.0,502.5,0.004153
196,21.500000,26.20,0.0,497.0,0.004153
197,21.500000,26.15,0.0,501.5,0.004145
...,...,...,...,...,...
6587,19.500000,26.20,0.0,466.0,0.003668
6588,19.426667,26.20,0.0,471.0,0.003651
6589,19.500000,26.20,0.0,471.0,0.003668
6590,19.390000,26.20,0.0,471.0,0.003643


In [39]:
df_high

,Temperature,Humidity,Light,CO2,HumidityRatio
1,23.18,27.2720,426.0,721.250000,0.004793
2,23.15,27.2675,429.5,714.000000,0.004783
3,23.15,27.2450,426.0,713.500000,0.004779
4,23.15,27.2000,426.0,708.250000,0.004772
5,23.10,27.2000,426.0,704.500000,0.004757
...,...,...,...,...,...
8139,21.05,36.0975,433.0,787.250000,0.005579
8140,21.05,35.9950,433.0,789.500000,0.005563
8141,21.10,36.0950,433.0,798.500000,0.005596
8142,21.10,36.2600,433.0,820.333333,0.005621


In [40]:
from scipy import stats

p_value = 0.05

rejected = 0

for col in df_1.columns:

    test = stats.ks_2samp(df_low[col], df_high[col])

    if test[1] < p_value:

         rejected += 1

         print("Column rejected", col)

print("We rejected",rejected,"columns in total")

Column rejected Temperature
Column rejected Humidity
Column rejected Light
Column rejected CO2
Column rejected HumidityRatio
We rejected 5 columns in total


In [9]:
pip install scikit-multiflow

Note: you may need to restart the kernel to use updated packages.


In [41]:
df_1

,Temperature,Humidity,Light,CO2,HumidityRatio
1,23.18,27.2720,426.0,721.250000,0.004793
2,23.15,27.2675,429.5,714.000000,0.004783
3,23.15,27.2450,426.0,713.500000,0.004779
4,23.15,27.2000,426.0,708.250000,0.004772
5,23.10,27.2000,426.0,704.500000,0.004757
...,...,...,...,...,...
8139,21.05,36.0975,433.0,787.250000,0.005579
8140,21.05,35.9950,433.0,789.500000,0.005563
8141,21.10,36.0950,433.0,798.500000,0.005596
8142,21.10,36.2600,433.0,820.333333,0.005621


In [42]:
t=df_1.columns

In [43]:
t=t[:len(t)-1]

In [44]:
k={}
c=0
from skmultiflow.drift_detection import ADWIN
adwin = ADWIN()
for col in t:
    data_stream=[]
    a = np.array(df_low[col])
    b = np.array(df_high[col])
    data_stream = np.concatenate((a,b))
  # Adding stream elements to ADWIN and verifying if drift occurred
    for i in range(len(data_stream)):
      adwin.add_element(data_stream[i])
      if adwin.detected_change():
        df_1.loc[i, 'Occupancy']=1
        c=c+1
    print('Run')
      
    k[col]=c

Run
Run
Run
Run


In [45]:
df_1[df_1['Occupancy']==1]

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
16,23.0000,27.1250,418.5,680.50,0.004715,1.0
18,22.9450,27.2900,0.0,685.00,0.004728,1.0
33,22.7000,27.5000,0.0,667.00,0.004694,1.0
48,22.3900,27.2900,0.0,639.00,0.004570,1.0
50,22.3900,27.3400,0.0,630.50,0.004579,1.0
...,...,...,...,...,...,...
7792,20.2900,33.0000,0.0,448.00,0.004861,1.0
7871,20.2000,33.0000,0.0,450.00,0.004834,1.0
7922,20.1000,33.0900,0.0,446.00,0.004817,1.0
7935,20.1000,33.0900,0.0,445.50,0.004817,1.0
